In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms

In [2]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20

# convert data to torch.FloatTensor
transform = transforms.ToTensor()

# choose the training and test datasets
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

100.0%


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100.0%
4.0%

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100.0%
100.0%


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [8]:
import torch.nn as nn
import torch.nn.functional as F

class kamnist(nn.Module):
    def __init__(self):
        super(kamnist, self).__init__()
        self.IMAGE_SIZE = 784
        self.FC2_SIZE = 512
        self.OUTPUT_SIZE = 10
        self.fc1 = nn.Linear(self.IMAGE_SIZE, self.FC2_SIZE)
        self.fc2 = nn.Linear(self.FC2_SIZE, self.FC2_SIZE)
        self.fc3 = nn.Linear(self.FC2_SIZE, self.OUTPUT_SIZE)
        self.dropout = nn.Dropout(0.2)
    def forward(self, x):
        # flatten x
        x = x.view(-1, self.IMAGE_SIZE)
        x = F.relu(self.fc1(x))

        return x
model = kamnist()
print(model)

kamnist(
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [10]:
EPOCHS = 35
model.train()
for EPOCH in range(EPOCHS):
    train_loss = 0.0
    for data, target in train_loader:
         optimizer.zero_grad()
         output = model(data)
         loss = criterion(output, target)
         loss.backward()
         optimizer.step()
         train_loss += loss.item()*data.size(0)

    # print training statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f}'.format(
        EPOCH+1, 
        train_loss
        ))

Epoch: 1 	Training Loss: 0.799465
Epoch: 2 	Training Loss: 0.412178
Epoch: 3 	Training Loss: 0.370368
Epoch: 4 	Training Loss: 0.349176
Epoch: 5 	Training Loss: 0.335669
Epoch: 6 	Training Loss: 0.326052
Epoch: 7 	Training Loss: 0.318737
Epoch: 8 	Training Loss: 0.312919
Epoch: 9 	Training Loss: 0.308142
Epoch: 10 	Training Loss: 0.304123
Epoch: 11 	Training Loss: 0.300679
Epoch: 12 	Training Loss: 0.297680
Epoch: 13 	Training Loss: 0.295036
Epoch: 14 	Training Loss: 0.292680
Epoch: 15 	Training Loss: 0.290563
Epoch: 16 	Training Loss: 0.288644
Epoch: 17 	Training Loss: 0.286894
Epoch: 18 	Training Loss: 0.285289
Epoch: 19 	Training Loss: 0.283808
Epoch: 20 	Training Loss: 0.282436
Epoch: 21 	Training Loss: 0.281159
Epoch: 22 	Training Loss: 0.279967
Epoch: 23 	Training Loss: 0.278850
Epoch: 24 	Training Loss: 0.277801
Epoch: 25 	Training Loss: 0.276811
Epoch: 26 	Training Loss: 0.275876
Epoch: 27 	Training Loss: 0.274991
Epoch: 28 	Training Loss: 0.274151
Epoch: 29 	Training Loss: 0.2